In [46]:
import pmdarima
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [47]:
# load the dataset
dataframe = pd.read_csv('archive/spx.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
dataset

array([[ 209.59],
       [ 210.88],
       [ 210.65],
       ...,
       [2699.63],
       [2716.31],
       [2718.37]], dtype=float32)

In [48]:
# split into train and test sets
train_size = int(len(dataset) * 0.7)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

5734 2458


In [49]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [50]:
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [51]:
trainX.shape

(5732, 1)

In [52]:
# trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
# testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [53]:
#automatically fit the optimal ARIMA model for given time series
arima_model_fitted = pmdarima.auto_arima(train)
# one-step out-of sample forecast
forecast =arima_model_fitted.predict(n_periods=1)[0]

# # load statsmodels
# import statsmodels.tsa.arima_model stm
# # fit ARIMA model
# model = stm.ARIMA(dataset, order=(3,1,2))
# model_fit = model.fit()
# # one-step out-of sample forecast
# forecast = model_fit.forecast()[0]

In [54]:
arima_model_fitted

ARIMA(order=(1, 1, 1), scoring_args={}, suppress_warnings=True)

In [55]:
model = pmdarima.arima.ARIMA(order=(1,1,1))
model_fit = model.fit(trainX)

In [34]:
trainPredict = model.predict(y=trainY,X=trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

TypeError: n_periods must be an int